<a href= "https://hrs.isr.umich.edu/sites/default/files/meta/2002/core/codebook/h02_00.html?_ga=2.14020593.714056361.1676427746-1610833755.1676427746">codebook</a>

In [2]:
import glob
import pandas as pd

all_dispositon_files = glob.glob('hrs_data/disposition/*.csv')
lst_disposition = []
for filename in all_dispositon_files:
    df_disposition = pd.read_csv(filename, index_col=None, header=0)
    lst_disposition.append(df_disposition)
dispositon_data = pd.concat(lst_disposition, axis=0,
                            ignore_index=True).iloc[:, :2]
dispositon_data['deceased'] = 1

In [8]:
all_core_files = glob.glob('hrs_data/*.csv')
lst_core = []
core_data = pd.DataFrame()
for filename in all_core_files:
    dfcore = pd.read_csv(filename, index_col=None, header=0)
    col_to_merge = dfcore.columns.difference(core_data.columns)
    core_data = pd.concat([core_data, dfcore[col_to_merge]], axis=1)
core_data

,HHID,PN,HSUBHH,GSUBHH,HPN_SP,HCSR,HFAMR,HFINR,HB001,HB002,...,HZ185,HZ186,HZ187,HZ188,HZ189,HZ190,HZ193,HZ194,HZ198,HZ199
0,3,10,0,0,20.0,5,5,1,1,NaN,...,NaN,9999996.0,NaN,99999996.0,NaN,999996.0,NaN,NaN,NaN,99999996.0
1,3,20,0,0,10.0,1,1,5,1,NaN,...,NaN,9999996.0,NaN,99999996.0,NaN,999996.0,NaN,NaN,NaN,99999996.0
2,10001,10,0,0,NaN,1,1,1,1,NaN,...,NaN,9999996.0,NaN,99999996.0,NaN,999996.0,NaN,NaN,NaN,99999996.0
3,10003,30,0,0,NaN,1,1,1,1,NaN,...,NaN,9999996.0,NaN,99999996.0,NaN,999996.0,1.0,12.0,NaN,99999996.0
4,10004,10,0,0,40.0,1,5,1,1,NaN,...,NaN,9999996.0,NaN,99999996.0,NaN,999996.0,NaN,NaN,500000.0,500000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18162,213471,10,0,0,NaN,1,1,1,1,NaN,...,NaN,9999996.0,NaN,99999996.0,NaN,999996.0,1.0,6.0,NaN,99999996.0
18163,213475,10,0,0,NaN,1,1,1,1,NaN,...,NaN,9999996.0,250000.0,250000.0,NaN,999996.0,1.0,3.0,NaN,99999996.0
18164,213479,10,0,0,20.0,5,5,1,1,NaN,...,NaN,9999996.0,NaN,99999996.0,NaN,999996.0,1.0,12.0,NaN,99999996.0
18165,213479,20,0,0,10.0,1,1,5,1,NaN,...,NaN,9999996.0,NaN,99999996.0,NaN,999996.0,1.0,24.0,NaN,99999996.0


In [9]:
df02_d = core_data.merge(dispositon_data, on=["HHID", "PN"], how="left")
df02_d['deceased'] = df02_d['deceased'].fillna(0)
df02_d['deceased'].value_counts()

0.0    17443
1.0      800
Name: deceased, dtype: int64